## Automatizacion de Migracion de datos de Access a PostgreSQL

In [1]:
!pip install pyodbc psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 9.6 MB/s eta 0:00:00


### Primero se realiza la coneccion a la base de datos MDB

In [2]:
import pyodbc

access_db_path = r'C:\user\Datos\basededatos.mdb'
access_connection = pyodbc.connect(
    f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};'
)
access_cursor = access_connection.cursor()

### Segundo se realiza la conexion con PostgreSQL

In [3]:
import psycopg2

pg_connection = psycopg2.connect(
    host="localhost",
    database="analisis_datos",
    user="user",
    password="password"
)
pg_cursor = pg_connection.cursor()

In [13]:
def check_postgres_connection(pg_connection):
    try:
        if pg_connection.closed == 0:
            print("La conexión a PostgreSQL es exitosa.")
        else:
            print("La conexión a PostgreSQL está cerrada.")
    except Exception as e:
        print(f"Error al verificar la conexión: {e}")


### Verificar conexion PostgreSQL

In [15]:
pg_cursor.execute("SELECT version()")
version = pg_cursor.fetchone()
print(f"Versión de PostgreSQL: {version}")


Versión de PostgreSQL: ('PostgreSQL 17.2 on x86_64-windows, compiled by msvc-19.42.34435, 64-bit',)


In [22]:
check_postgres_connection(pg_connection)


La conexión a PostgreSQL es exitosa.


### Obtener la estructura de access

In [23]:
def get_access_table_structure(cursor, table_name):
    cursor.execute(f"SELECT * FROM {table_name} WHERE 1=0")  # No obtiene registros, solo la estructura
    columns = cursor.description
    structure = []
    for column in columns:
        name = column[0]
        # Mapear tipos de Access a tipos equivalentes en PostgreSQL
        access_type = "TEXT"  # Como ejemplo, ajustar según el tipo de dato real
        if "int" in column[1].__name__:
            access_type = "INTEGER"
        elif "float" in column[1].__name__:
            access_type = "FLOAT"
        elif "date" in column[1].__name__:
            access_type = "TIMESTAMP"
        structure.append((name, access_type))
    return structure


### Crear la Tabla en PostgreSQL

In [24]:
def create_postgres_table(pg_cursor, table_name, structure):
    columns_definition = ", ".join([f'"{col_name}" {col_type}' for col_name, col_type in structure])
    create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns_definition});'
    
    try:
        pg_cursor.execute(create_table_query)
    except Exception as e:
        print(f"Error al crear la tabla '{table_name}': {e}")



### Integracion de estructura de tablas

In [25]:
access_table_name = "analisis_ventas"
postgres_table_name = access_table_name  # Puedes cambiar el nombre si es necesario

# Obtiene la estructura desde Access
structure = get_access_table_structure(access_cursor, access_table_name)

# Crea la tabla en PostgreSQL
create_postgres_table(pg_cursor, postgres_table_name, structure)
pg_connection.commit()
print(f"Tabla '{postgres_table_name}' creada exitosamente en PostgreSQL.")


Tabla 'analisis_ventas' creada exitosamente en PostgreSQL.


In [26]:
def count_records_in_postgres(cursor, table_name):
    try:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        return cursor.fetchone()[0]
    except Exception as e:
        print(f"Error al contar registros en la tabla '{table_name}': {e}")
        return None


In [29]:
def clean_null_characters(row):
    # Reemplaza los caracteres nulos (\x00) por un valor válido (por ejemplo, una cadena vacía o None)
    return [None if isinstance(value, str) and '\x00' in value else value for value in row]

def migrate_table(access_cursor, pg_cursor, pg_connection, table_name):
    try:
        access_cursor.execute(f"SELECT * FROM {table_name}")
        columns = [column[0] for column in access_cursor.description]  # Nombres de las columnas
        column_names = ', '.join([f'"{col}"' for col in columns])  # Poner entre comillas los nombres de las columnas
        placeholders = ', '.join(['%s'] * len(columns))

        # Leer registros desde Access
        rows = access_cursor.fetchall()

        # Limpiar caracteres nulos de los datos antes de insertarlos
        rows = [clean_null_characters(row) for row in rows]

        # Limpiar la tabla antes (opcional)
        pg_cursor.execute(f"TRUNCATE TABLE \"{table_name}\" RESTART IDENTITY;")  # También comillas dobles para la tabla
        
        # Preparar consulta de inserción
        insert_query = f"INSERT INTO \"{table_name}\" ({column_names}) VALUES ({placeholders})"
        
        for row in rows:
            pg_cursor.execute(insert_query, row)
        
        pg_connection.commit()
        print(f"Tabla {table_name} migrada exitosamente.")

    except Exception as e:
        pg_connection.rollback()  # Restaura el estado si ocurre un error
        print(f"Error al migrar la tabla {table_name}: {e}")

# Migrar una tabla específica
table_name = "analisis_ventas"
migrate_table(access_cursor, pg_cursor, pg_connection, table_name)


Tabla analisis_ventas migrada exitosamente.


#### Contar Registros en Access

In [30]:
def count_records_in_access(cursor, table_name):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    return cursor.fetchone()[0]

# Llama a la función y almacena tanto el nombre como el conteo
table_name = "analisis_ventas"
access_count = count_records_in_access(access_cursor, table_name)

# Incluye el nombre de la tabla directamente en el print
print(f"Registros en Access (tabla '{table_name}'): {access_count}")

Registros en Access (tabla 'analisis_ventas'): 1367579


In [31]:
# Función para contar registros en PostgreSQL con manejo de excepciones
def count_records_in_postgres(cursor, table_name):
    try:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        return cursor.fetchone()[0]
    except Exception as e:
        print(f"Error al contar registros en la tabla '{table_name}': {e}")
        return None

# Comprobar conexión antes de contar registros
def check_postgres_connection(pg_connection):
    try:
        if pg_connection.closed == 0:
            print("La conexión a PostgreSQL es exitosa.")
        else:
            print("La conexión a PostgreSQL está cerrada.")
    except Exception as e:
        print(f"Error al verificar la conexión: {e}")

# Llama a la función para verificar la conexión
check_postgres_connection(pg_connection)

# Llama a la función para contar registros en una tabla específica
table_name = "analisis_ventas"
postgres_count = count_records_in_postgres(pg_cursor, table_name)
if postgres_count is not None:
    print(f"Registros en PostgreSQL (tabla '{table_name}'): {postgres_count}")
else:
    print(f"No se pudo obtener el conteo de registros para la tabla '{table_name}'.")


La conexión a PostgreSQL es exitosa.
Registros en PostgreSQL (tabla 'analisis_ventas'): 1367579


### Validar el contenido de los registros

In [35]:
def clean_null_characters(row):
    # Reemplaza los caracteres nulos (\x00) por un valor válido (por ejemplo, una cadena vacía o None)
    return [None if isinstance(value, str) and '\x00' in value else value for value in row]

# Seleccionar un registro de Access
access_cursor.execute(f"SELECT TOP 1 * FROM {table_name}")
access_row = access_cursor.fetchone()
print("Registro en Access:", access_row)

# Limpiar caracteres nulos en el registro de Access
access_row = clean_null_characters(access_row)

# Seleccionar el mismo registro de PostgreSQL
pg_cursor.execute(f"SELECT * FROM \"{table_name}\" LIMIT 1")  # Asegúrate de usar comillas dobles para el nombre de la tabla
pg_row = pg_cursor.fetchone()
print("Registro en PostgreSQL:", pg_row)

# Limpiar caracteres nulos en el registro de PostgreSQL
pg_row = clean_null_characters(pg_row)

# Comparar los registros
if access_row == pg_row:
    print("El contenido del registro es idéntico.")
else:
    print("Diferencias encontradas en los datos. Revisa la migración.")



Registro en Access: ('ESTE', 'TAMANACO', 'TAMANACO A8', 'T2961', 18000034992.0, 'RES EL MADRIGAL', 9000055759.0, 5000000026.0, 0, 101345229.0, 2006, datetime.datetime(2006, 5, 5, 0, 0), 'C5000001', 15, 0, 120, 208, 1002, 'PN5D-G', 'KWH 2X120/2082X120/240V15/120A', None, 100000271675.0, 6000358532.0, 'JUAN L OLIVER P', 7001711886.0, datetime.datetime(2002, 5, 13, 0, 0), datetime.datetime(9999, 12, 31, 0, 0), 'CICNOR18', 4, 'RE01', 1010, 'E0_RES', 1133, 0, 0, 'SU100040', 680, 'SRA71014', 0.0, '53EM0362', 154, 445, 596, 649, 147, 75, 435, 882, 986, 868, 876, 638, -2.140873670578003, 18.76720428466797, 563, Decimal('85.4722'), 4.965517044067383, 15.741935729980469, 20.516128540039062, 21.0, 2.433333396911621, 2.4193549156188965, 15.741935729980469, 31.482759475708008, 31.84375, 28.5, 28.20689582824707, 22.354839324951172, -0.6539325714111328, -0.27744102478027344, -0.05105289816856384, 3.272547721862793, 1.025333285331726, -0.8275862336158752, -0.5227122902870178, -0.07565924525260925, 0.0

In [36]:
# Obtener la estructura de la tabla en PostgreSQL
pg_cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = %s", (table_name,))
columns_pg = pg_cursor.fetchall()
print("Estructura de la tabla en PostgreSQL:", columns_pg)


Estructura de la tabla en PostgreSQL: [('kva_08', 'integer'), ('kva_09', 'integer'), ('kva_10', 'integer'), ('kva_11', 'integer'), ('kva_12', 'integer'), ('kva_13', 'integer'), ('Num_Fases', 'integer'), ('Pto_Suministro', 'double precision'), ('Instalacion', 'double precision'), ('Edo_Inst', 'integer'), ('Cta_Contrato', 'double precision'), ('Int_Comercial', 'double precision'), ('Medidor', 'double precision'), ('Contrato', 'double precision'), ('Fecha_Alta', 'timestamp without time zone'), ('Fecha_Baja', 'timestamp without time zone'), ('Certificacion', 'integer'), ('Retail', 'integer'), ('Fec_Montaje', 'timestamp without time zone'), ('CNAE', 'integer'), ('Obj_Conex', 'double precision'), ('Ctro_Emp', 'integer'), ('CTC', 'integer'), ('DAC', 'integer'), ('AMP_PRI', 'integer'), ('REGPOLIT', 'integer'), ('AMP_SEC', 'integer'), ('CASAS', 'double precision'), ('VOL_PRI', 'integer'), ('CSMODIF_01', 'integer'), ('CSMODIF_02', 'integer'), ('Motivo_garantia', 'integer'), ('Ultimo_Contrato', '